# 06 — Regularization: Ridge, Lasso, Elastic Net

**Goal:** Understand shrinkage vs sparsity; tune λ; compare coefficient paths.


In [ ]:
import warnings
import numpy as np
import pandas as pd

from sklearn.datasets import fetch_california_housing, make_regression
from sklearn.model_selection import train_test_split

def load_regression_data(random_state=42):
    """Return (X, y, feature_names) as numpy arrays.
    Try California Housing; fallback to synthetic if unavailable (e.g., offline).
    """
    try:
        cali = fetch_california_housing(as_frame=True)
        df = cali.frame.copy()
        X = df.drop(columns=["MedHouseVal"]).values
        y = df["MedHouseVal"].values
        feature_names = list(df.drop(columns=["MedHouseVal"]).columns)
    except Exception as e:
        warnings.warn(f"California Housing fetch failed: {e}. Falling back to synthetic make_regression.")
        X, y = make_regression(n_samples=5000, n_features=8, n_informative=6, noise=8.5, random_state=random_state)
        feature_names = [f"x{i}" for i in range(X.shape[1])]
    return X, y, feature_names

def train_val_test_split(X, y, random_state=42):
    # 60/20/20 split: train/val/test
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=random_state)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=random_state)
    return (X_train, y_train), (X_val, y_val), (X_test, y_test)

def rmse(y_true, y_pred):
    return float(np.sqrt(np.mean((y_true - y_pred)**2)))

def mae(y_true, y_pred):
    return float(np.mean(np.abs(y_true - y_pred)))

def r2(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return float(1 - ss_res/ss_tot)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import KFold, cross_val_score

X, y, feature_names = load_regression_data()
(X_train, y_train), (X_val, y_val), (X_test, y_test) = train_val_test_split(X, y)

alphas = np.logspace(-3, 3, 13)

def cv_rmse(model, X, y):
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, scoring="neg_root_mean_squared_error", cv=cv, n_jobs=-1)
    return -scores

ridge_rmse = []
lasso_rmse = []
enet_rmse = []

for a in alphas:
    ridge = Pipeline([("scaler", StandardScaler()), ("model", Ridge(alpha=a))])
    lasso = Pipeline([("scaler", StandardScaler()), ("model", Lasso(alpha=a, max_iter=5000))])
    enet  = Pipeline([("scaler", StandardScaler()), ("model", ElasticNet(alpha=a, l1_ratio=0.5, max_iter=5000))])
    ridge_rmse.append(cv_rmse(ridge, X_train, y_train).mean())
    lasso_rmse.append(cv_rmse(lasso, X_train, y_train).mean())
    enet_rmse.append(cv_rmse(enet, X_train, y_train).mean())

plt.figure()
plt.semilogx(alphas, ridge_rmse, marker="o")
plt.title("Ridge: alpha vs CV RMSE")
plt.xlabel("alpha"); plt.ylabel("RMSE")

plt.figure()
plt.semilogx(alphas, lasso_rmse, marker="o")
plt.title("Lasso: alpha vs CV RMSE")
plt.xlabel("alpha"); plt.ylabel("RMSE")

plt.figure()
plt.semilogx(alphas, enet_rmse, marker="o")
plt.title("ElasticNet: alpha vs CV RMSE (l1_ratio=0.5)")
plt.xlabel("alpha"); plt.ylabel("RMSE")


In [ ]:
# TODO: Fit best Ridge and Lasso and plot sorted absolute coefficients side by side; discuss sparsity vs shrinkage.
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, Lasso

scaler = StandardScaler().fit(X_train)
Xtr_s = scaler.transform(X_train)
Xval_s = scaler.transform(X_val)

best_alpha_ridge = float(alphas[np.argmin(ridge_rmse)])
best_alpha_lasso = float(alphas[np.argmin(lasso_rmse)])

ridge_best = Ridge(alpha=best_alpha_ridge).fit(Xtr_s, y_train)
lasso_best = Lasso(alpha=best_alpha_lasso, max_iter=5000).fit(Xtr_s, y_train)

coef_ridge = np.abs(ridge_best.coef_)
coef_lasso = np.abs(lasso_best.coef_)

print("Best alpha (Ridge):", best_alpha_ridge, "Best alpha (Lasso):", best_alpha_lasso)
